# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [2]:
pip install pandas

     |████████████████████████████████| 11.7 MB 1.5 MB/s eta 0:00:01
You should consider upgrading via the '/Applications/Mu Editor.app/Contents/Resources/Python/bin/Mu Editor -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd 
import requests 
import math
from scipy import stats
import yfinance as yf

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [3]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks
tickers = stocks['Ticker']

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [4]:
ratios = {}
for ticker in tickers:
    try:
       ratios[ticker] = (yf.Ticker(ticker).info['trailingPE'])    
    except:
        continue
ratios.items


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=uurKildAfsn
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=uurKildAfsn
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DISH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DISH&crumb=uurKildAfsn
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FLT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.c

<function dict.items>

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

34.91

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,99.160,46.39,N/A
1,AAL,13.430,-1.65,N/A
2,AAP,156.890,28.9,N/A
3,AAPL,475.040,34.8,N/A
4,ABBV,95.770,20.14,N/A
...,...,...,...,...
500,YUM,93.960,28.5,N/A
501,ZBH,143.010,723.05,N/A
502,ZBRA,299.348,31.74,N/A
503,ZION,34.700,13.44,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [5]:
price = {}
for ticker in tickers:
    try:
       price[ticker] = (yf.Ticker(ticker).info['previousClose'])    
    except:
        continue


404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ABC?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ABC&crumb=uurKildAfsn
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/ATVI?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=ATVI&crumb=uurKildAfsn
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/DISH?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=DISH&crumb=uurKildAfsn
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/FLT?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.c

In [6]:
my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
value_df = pd.DataFrame(columns=my_columns)

for key, value in ratios.items():
    # Create a dictionary for the new row
    new_row = {
        'Ticker': key,
        'Price': price[key],  
        'Price-to-Earnings Ratio': value,
        'Number of Shares to Buy': 0
    }
    # Append the new row to the DataFrame
    value_df = pd.concat([value_df, pd.DataFrame([new_row])], ignore_index=True)

print(value_df)

    Ticker   Price  Price-to-Earnings Ratio Number of Shares to Buy
0        A  139.49                33.427895                       0
1      AAP   63.09               175.916670                       0
2     AAPL  218.80                34.538105                       0
3     ABBV  186.78                61.979935                       0
4     ABMD  381.02                65.355060                       0
..     ...     ...                      ...                     ...
426    YUM  132.17                23.551420                       0
427    ZBH  111.37                24.154013                       0
428   ZBRA  348.12                78.919106                       0
429   ZION   51.91                12.982411                       0
430    ZTS  181.83                34.756756                       0

[431 rows x 4 columns]


/var/folders/q9/8hz1nyl113q8w927cy4s5dkh0000gn/T/ipykernel_54616/3683192312.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  value_df = pd.concat([value_df, pd.DataFrame([new_row])], ignore_index=True)


In [12]:
value_df.sort_values('Price-to-Earnings Ratio',ascending=True,inplace=True)
value_df = value_df[:50]


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [13]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [14]:
portfolio_input()

You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [17]:
NumShares = []
position_size = int(portfolio_size)/50
for price in value_df['Price']:
    NumShares.append(math.floor(position_size/price))
value_df['Number of Shares to Buy']=NumShares
value_df


/var/folders/q9/8hz1nyl113q8w927cy4s5dkh0000gn/T/ipykernel_54616/2173356228.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  value_df['Number of Shares to Buy']=NumShares


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
327,PRGO,28.29,2827.000000,70
168,GILD,78.05,211.277760,25
249,LUV,27.04,207.230770,73
403,VNO,29.53,199.933320,67
1,AAP,63.09,175.916670,31
27,AMD,138.44,172.000000,14
204,INCY,67.79,158.707320,29
212,IRM,101.29,155.393940,19
291,NOW,798.80,147.802170,2
41,ARE,119.02,144.802470,16


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,100.010,N/A,46.95,N/A,6.596140,N/A,26.372976,N/A,11.311629,N/A,N/A
1,AAL,13.360,N/A,-1.71,N/A,-60.417952,N/A,5.952664,N/A,3.098483,N/A,N/A
2,AAP,161.440,N/A,29,N/A,3.123759,N/A,15.086368,N/A,3.208667,N/A,N/A
3,AAPL,466.070,N/A,34.69,N/A,22.373999,N/A,25.708773,N/A,19.428993,N/A,N/A
4,ABBV,96.680,N/A,21,N/A,-21.463532,N/A,12.272585,N/A,7.672427,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.320,N/A,28,N/A,-3.659682,N/A,18.841249,N/A,13.891510,N/A,N/A
501,ZBH,143.470,N/A,718.1,N/A,2.390128,N/A,17.170711,N/A,7.478460,N/A,N/A
502,ZBRA,288.222,N/A,31.86,N/A,8.600669,N/A,19.480804,N/A,8.449885,N/A,N/A
503,ZION,35.770,N/A,13.24,N/A,0.766237,N/A,NaN,N/A,NaN,N/A,N/A


{'week52change': 0.290414, 'week52high': 169.51, 'week52low': 90.41, 'marketcap': 75861250162, 'employees': 10724, 'day200MovingAvg': 134.17, 'day50MovingAvg': 148.64, 'float': 479955020, 'avg10Volume': 1641586.3, 'avg30Volume': 1479579.2, 'ttmEPS': 3.499, 'ttmDividendRate': 0.77, 'companyName': 'Zoetis, Inc.', 'sharesOutstanding': 494883276, 'maxChangePercent': 3.3525, 'year5ChangePercent': 1.69, 'year2ChangePercent': 0.608, 'year1ChangePercent': 0.281583, 'ytdChangePercent': 0.189876, 'month6ChangePercent': 0.10241, 'month3ChangePercent': 0.297048, 'month1ChangePercent': 0.159224, 'day30ChangePercent': 0.165634, 'day5ChangePercent': -0.012817, 'nextDividendDate': None, 'dividendYield': 0.00489706569361443, 'nextEarningsDate': '2020-10-13', 'exDividendDate': '2020-07-01', 'peRatio': 49.1, 'beta': 0.9584403132945124, 'totalCash': 1974321215, 'currentDebt': 549171552, 'revenue': 6518431250, 'grossProfit': 4133660573, 'totalRevenue': 6300683466, 'EBITDA': 2502772553, 'revenuePerShare': 1

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
17,AFL,39.060,N/A,9.7,N/A,0.950934,N/A,NaN,N/A,NaN,N/A,N/A
18,AIG,31.950,N/A,-5.88,N/A,0.407454,N/A,NaN,N/A,NaN,N/A,N/A
20,AIZ,127.680,N/A,20.22,N/A,1.360264,N/A,NaN,N/A,NaN,N/A,N/A
26,ALL,97.880,N/A,7.1,N/A,1.173772,N/A,NaN,N/A,NaN,N/A,N/A
39,ANTM,293.560,N/A,12.24,N/A,2.304037,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,196.350,N/A,26.09,N/A,13.486066,N/A,17.533688,N/A,NaN,N/A,N/A
56,BAC,26.970,N/A,12.63,N/A,0.867662,N/A,NaN,N/A,NaN,N/A,N/A
64,BK,38.160,N/A,8.27,N/A,0.804550,N/A,NaN,N/A,NaN,N/A,N/A
65,BKNG,1866.170,N/A,30.78,N/A,12.571484,N/A,12.859665,N/A,NaN,N/A,N/A
75,C,55.140,N/A,9.52,N/A,0.576968,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

0      0.841584
1      0.112871
2      0.623762
3      0.740594
4      0.427723
         ...   
500         0.6
501    0.994059
502    0.693069
503    0.257426
504    0.843564
Name: PE Percentile, Length: 505, dtype: object
0       0.752475
1      0.0158416
2       0.510891
3       0.940594
4      0.0257426
         ...    
500     0.049505
501     0.415842
502     0.811881
503     0.132673
504     0.956436
Name: PB Percentile, Length: 505, dtype: object
0       0.877228
1      0.0732673
2        0.50099
3       0.861386
4       0.350495
         ...    
500     0.744554
501     0.572277
502     0.762376
503      0.69505
504     0.924752
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.552475
1      0.0574257
2      0.0653465
3       0.865347
4       0.340594
         ...    
500     0.744554
501     0.326733
502          0.4
503     0.644554
504     0.869307
Name: EV/GP Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,100.010,N/A,46.95,0.841584,6.596140,0.752475,26.372976,0.877228,11.311629,0.552475,N/A
1,AAL,13.360,N/A,-1.71,0.112871,-60.417952,0.0158416,5.952664,0.0732673,3.098483,0.0574257,N/A
2,AAP,161.440,N/A,29.00,0.623762,3.123759,0.510891,15.086368,0.50099,3.208667,0.0653465,N/A
3,AAPL,466.070,N/A,34.69,0.740594,22.373999,0.940594,25.708773,0.861386,19.428993,0.865347,N/A
4,ABBV,96.680,N/A,21.00,0.427723,-21.463532,0.0257426,12.272585,0.350495,7.672427,0.340594,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.320,N/A,28.00,0.6,-3.659682,0.049505,18.841249,0.744554,13.891510,0.744554,N/A
501,ZBH,143.470,N/A,718.10,0.994059,2.390128,0.415842,17.170711,0.572277,7.478460,0.326733,N/A
502,ZBRA,288.222,N/A,31.86,0.693069,8.600669,0.811881,19.480804,0.762376,8.449885,0.4,N/A
503,ZION,35.770,N/A,13.24,0.257426,0.766237,0.132673,18.729176,0.69505,12.206556,0.644554,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,100.010,N/A,46.95,0.841584,6.596140,0.752475,26.372976,0.877228,11.311629,0.552475,0.755941
1,AAL,13.360,N/A,-1.71,0.112871,-60.417952,0.0158416,5.952664,0.0732673,3.098483,0.0574257,0.0648515
2,AAP,161.440,N/A,29.00,0.623762,3.123759,0.510891,15.086368,0.50099,3.208667,0.0653465,0.425248
3,AAPL,466.070,N/A,34.69,0.740594,22.373999,0.940594,25.708773,0.861386,19.428993,0.865347,0.85198
4,ABBV,96.680,N/A,21.00,0.427723,-21.463532,0.0257426,12.272585,0.350495,7.672427,0.340594,0.286139
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.320,N/A,28.00,0.6,-3.659682,0.049505,18.841249,0.744554,13.891510,0.744554,0.534653
501,ZBH,143.470,N/A,718.10,0.994059,2.390128,0.415842,17.170711,0.572277,7.478460,0.326733,0.577228
502,ZBRA,288.222,N/A,31.86,0.693069,8.600669,0.811881,19.480804,0.762376,8.449885,0.4,0.666832
503,ZION,35.770,N/A,13.24,0.257426,0.766237,0.132673,18.729176,0.69505,12.206556,0.644554,0.432426


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

Enter the value of your portfolio:1000000


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,HPE,9.830,1994,-293.68,0.0118812,0.768699,0.134653,4.757379,0.0455446,2.575722,0.039604,0.0579208
1,FTI,8.870,2210,-0.68,0.134653,0.526579,0.0732673,2.722313,0.00792079,1.745265,0.0178218,0.0584158
2,AAL,13.360,1467,-1.71,0.112871,-60.417952,0.0158416,5.952664,0.0732673,3.098483,0.0574257,0.0648515
3,CCL,15.610,1256,-3.90,0.10297,0.380999,0.0554455,3.751649,0.0237624,3.458441,0.0811881,0.0658416
4,HFC,25.660,764,-26.18,0.0435644,0.716944,0.120792,3.301222,0.0118812,3.693071,0.0891089,0.0663366
5,HPQ,19.140,1024,9.17,0.172277,-23.323972,0.0237624,5.822424,0.0673267,2.516859,0.0356436,0.0747525
6,XRX,17.800,1101,9.35,0.174257,0.640297,0.0990099,3.620185,0.019802,1.604068,0.0138614,0.0767327
7,TPR,16.010,1224,-20.58,0.049505,1.200922,0.231683,3.870569,0.029703,1.149885,0.0019802,0.0782178
8,LB,27.330,717,-10.49,0.0633663,-5.047485,0.0455446,8.144139,0.170297,3.119623,0.0613861,0.0851485
9,SYF,25.300,775,7.77,0.152475,0.971730,0.180198,2.129599,0.0039604,1.604512,0.0158416,0.0881188


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

## Saving Our Excel Output
As before, saving our Excel output is very easy: